# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!

In this lab, you'll practice your scraping skills on an online music magazine and events website called Resident Advisor.

## Objectives

You will be able to:

* Create a full scraping pipeline that involves traversing over many pages of a website, dealing with errors and storing data

## View the Website

For this lab, you'll be scraping the https://ra.co website. For reproducibility we will use the [Internet Archive](https://archive.org/) Wayback Machine to retrieve a version of this page from March 2019.

Start by navigating to the events page [here](https://web.archive.org/web/20210325230938/https://ra.co/events/us/newyork?week=2019-03-30) in your browser. It should look something like this:

<img src="images/ra_top.png">

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

## Write a Function to Scrape all of the Events on the Given Page

The function should return a Pandas DataFrame with columns for the `Event_Name`, `Venue`, and `Number_of_Attendees`.

Start by importing the relevant libraries, making a request to the relevant URL, and exploring the contents of the response with `BeautifulSoup`. Then fill in the `scrape_events` function with the relevant code.

In [135]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [74]:
EVENTS_PAGE_URL = "https://web.archive.org/web/20210326225933/https://ra.co/events/us/newyork?week=2019-03-30"
r = requests.get(EVENTS_PAGE_URL)
soup = BeautifulSoup(r.content,'html.parser')

In [76]:
# Find the container with event listings in it
# Some hints are giving along the way

# This page is organized somewhat unusually, and many of
# the CSS attributes seem auto-generated. We notice that
# there is a div with "events-all" in its attributes that
# looks promising if we use soup.find(), call this events_all_div

events_all_div = soup.find('div',attrs={"data-tracking-id": "events-all"})

# The actual content is nested in a ul containing a single
# li within that div. Unclear why they are using a "list"
# concept for one element, but let's go ahead and select it
# Call this event_listings and use it to find ul and li in 
# events_all_div

event_listings = events_all_div.find('ul').find('li')


# Print out some chunks of the text inside to make sure we
# have everything we need in here
# For example print the events for March 30th and 31st


In [78]:
# Find a list of events by date within that container

# Now we look at what is inside of that event_listings li tag.
# Based on looking at the HTML with developer tools, we see
# that there are 13 children of that tag, all divs. Each div
# is either a container of events on a given date, or empty

# Let's create a collection of those divs. recursive=False
# means we stop at 1 level below the event_listings li
dates = event_listings.findChildren(recursive=False)

# Now let's print out the start of the March 30th and March
# 31st sections again. This time each is in its own "date"
# container

# March 30th is at the 0 index
print("0 index:", dates[0].text[:200])
print()
# The 1 index is empty. We'll need to skip this later
print("1 index: ", dates[1].text)
print()
# March 31st is at the 2 index
print("2 index:", dates[2].text[:200])

# Now we know we can loop over all of the items in the dates
# list of divs to find the dates, although some will be blank
# so we'll need to skip them

0 index: ̸Sat, 30 MarUnterMania IIMary Yuzovskaya, Manni Dee, Umfang, Juana, The Lady MachineTBA - New YorkRARA Tickets457Cocoon New York: Sven Väth, Ilario Alicante, Butch & TaimurSven Vath, Butch, Taimur, Il

1 index:  

2 index: ̸Sun, 31 MarSunday: Soul SummitNowadaysRARA Tickets132New Dad & Aaron Clark (Honcho)Aaron Clark, New DadAce Hotel3ParadiscoOccupy The DiscoLe Bain3Sunday Soiree: Unknown Showcase (Detroit)Ryan Dahl, H


In [91]:
# Extract the date (e.g. Sat, 30 Mar) from one of those containers
# Call this first_date

# Grabbing just one to practice on
first_date = dates[0]

# This div contains a div with the date, followed by several uls
# containing actual event information

# The div with the date happens to have another human-readable
# CSS class, so let's use that to select it then grab its text
# Call this date, and use class_=sticky header as an argument for
# first_date.find
date = first_date.find('div', class_='sticky-header').text[1:]

In [133]:
test = first_date.findAll('ul')[0].findAll(attrs={"height": 30})[1]
print(test.prettify())

<div class="Box-omzyfs-0 sc-AxjAm jYikCb" height="30">
 <div class="Box-omzyfs-0 sc-AxjAm faxXyr" color="secondary" height="24" width="24">
  <svg aria-label="Ticket" height="100%" viewbox="0 0 24 24" width="100%">
   <g fill="none" fill-rule="evenodd">
    <path d="M0 0h24v24H0z" fill="none">
    </path>
    <path d="M8.243 6.827a1 1 0 010-1.414l2.12-2.12a1 1 0 011.415 0l8.485 8.484a1 1 0 010 1.414l-2.12 2.121a1 1 0 01-1.415 0 1 1 0 00-1.415 1.414 1 1 0 010 1.414l-2.12 2.123a1 1 0 01-1.415 0l-8.485-8.486a1 1 0 010-1.414l2.122-2.121a1 1 0 011.414 0 1.001 1.001 0 001.414-1.415zm2-.586a3.002 3.002 0 01-4 4.002l-.829.828 7.07 7.07.829-.828a3.002 3.002 0 014.001-4.001l.828-.828-7.071-7.07-.828.827z" fill="currentColor">
    </path>
   </g>
  </svg>
 </div>
 <a class="Link__AnchorWrapper-k7o46r-1 cIDdag" color="secondary" href="/web/20210326225933/https://ra.co/events/1234892#tickets">
  <span class="Text-sc-1t0gn2o-0 Link__StyledLink-k7o46r-0 iHZuSI" color="secondary" font-weight="normal" 

In [112]:
# Extract the name, venue, and number of attendees from one of the
# events within that container

# As noted previously, the div with information about events on
# this date contains several ul tags, each with information about
# a specific event. Get a list of them.
# (Again this is an odd use of HTML, to have an unordered list
# containing a single list item. But we scrape what we find!)
first_date_events = first_date.findAll('ul')

# Grabbing the first event ul to practice on
first_event = first_date_events[0]

# Each event ul contains a single h3 with the event name, easy enough
name = first_date_events[0].find('h3')

# First, get all 1-3 divs that match this description,
# where first_event.findAll has attrs={"height": 30}
# as one of its arguments
venue_and_attendees = first_event.findAll(attrs={"height": 30})
# The venue is the 0th (left-most) div, get its text
venue = venue_and_attendees[0].text
# The number of attendees is the last div (although it's sometimes
# missing), get its text
num_attendees = venue_and_attendees[2].text

In [136]:
# Run the code below
# Make sure you understand it since it will
# for the basis of the definition of scrape_events below

# Create an empty list to hold results
rows = []

# Loop over all date containers on the page
for date_container in dates:
    
    # First check if this is one of the empty divs. If it is,
    # skip ahead to the next one
    if not date_container.text:
        continue
    
    # Same logic as above to extract the date
    date = date_container.find("div", class_="sticky-header").text
    date = date.strip("'̸")
    
    # This time, loop over all of the events
    events = date_container.findChildren("ul")
    for event in events:
        
        # Same logic as above to extract the name, venue, attendees
        name = event.find("h3").text
        venue_and_attendees = event.findAll("div", attrs={"height": 30})
        venue = venue_and_attendees[0].text
        try:
            num_attendees = int(venue_and_attendees[-1].text)
        except ValueError:
            num_attendees = np.nan
            
        # New piece here: appending the new information to rows list
        rows.append([name, venue, date, num_attendees])

# Make the list of lists into a dataframe and display
df = pd.DataFrame(rows)
df  

,0,1,2,3
0,UnterMania II,TBA - New York,"Sat, 30 Mar",457.0
1,"Cocoon New York: Sven Väth, Ilario Alicante, B...",99 Scott Ave,"Sat, 30 Mar",407.0
2,Horse Meat Disco - New York Residency,Elsewhere,"Sat, 30 Mar",375.0
3,Rave: Underground Resistance All Night,Nowadays,"Sat, 30 Mar",232.0
4,"Believe You Me // Beta Librae, Stephan Kimbel,...",TBA - New York,"Sat, 30 Mar",89.0
...,...,...,...,...
114,A Night at the Baths,C'mon Everybody,"Fri, 5 Apr",1.0
115,Blaqk Audio,Music Hall of Williamsburg,"Fri, 5 Apr",1.0
116,Erik the Lover,Erv's,"Fri, 5 Apr",1.0
117,Wax On Vissions,Starliner,"Fri, 5 Apr",1.0


In [137]:
# Bring it all together in a function that makes the request, gets the
# list of entries from the response, loops over that list to extract the
# name, venue, date, and number of attendees for each event, and returns
# that list of events as a dataframe

def scrape_events(events_page_url):
    r = requests.get(events_page_url)
    soup = BeautifulSoup(r.content,'html.parser')
    events_all_div = soup.find('div',attrs={"data-tracking-id": "events-all"})
    event_listings = events_all_div.find('ul').find('li')
    dates = event_listings.findChildren(recursive=False)
    rows = []
    for date_container in dates:
        if not date_container.text:
            continue
        date = date_container.find("div", class_="sticky-header").text
        date = date.strip("'̸")
        events = date_container.findChildren("ul")
        for event in events:
            name = event.find("h3").text
            venue_and_attendees = event.findAll("div", attrs={"height": 30})
            venue = venue_and_attendees[0].text
            try:
                num_attendees = int(venue_and_attendees[-1].text)
            except ValueError:
                num_attendees = np.nan
            rows.append([name, venue, date, num_attendees])
    df = pd.DataFrame(rows)    
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

In [138]:
# Test out your function
scrape_events(EVENTS_PAGE_URL)

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,UnterMania II,TBA - New York,"Sat, 30 Mar",457.0
1,"Cocoon New York: Sven Väth, Ilario Alicante, B...",99 Scott Ave,"Sat, 30 Mar",407.0
2,Horse Meat Disco - New York Residency,Elsewhere,"Sat, 30 Mar",375.0
3,Rave: Underground Resistance All Night,Nowadays,"Sat, 30 Mar",232.0
4,"Believe You Me // Beta Librae, Stephan Kimbel,...",TBA - New York,"Sat, 30 Mar",89.0
...,...,...,...,...
114,A Night at the Baths,C'mon Everybody,"Fri, 5 Apr",1.0
115,Blaqk Audio,Music Hall of Williamsburg,"Fri, 5 Apr",1.0
116,Erik the Lover,Erv's,"Fri, 5 Apr",1.0
117,Wax On Vissions,Starliner,"Fri, 5 Apr",1.0


## Write a Function to Retrieve the URL for the Next Page

As you scroll down, there should be a button labeled "Next Week" that will take you to the next page of events. Write code to find that button and extract the URL from it.

This is a relative path, so make sure you add `https://web.archive.org` to the front to get the URL.

![next page](images/ra_next.png)

In [160]:
test = soup.find('svg',attrs={"aria-label": "Right arrow"}).parent.previous_sibling['href']
print(test)

/web/20210326225933/https://ra.co/events/us/newyork?week=2019-04-06


In [161]:
# Find the button, find the relative path, create the URL for the current `soup`

# This is tricky again, since there are not a lot of
# human-readable CSS classes

# One unique thing we notice is a > icon on the part where
# you click to go to the next page. It's an SVG with an 
# aria-label of "Right arrow", this soup.find() will have
# attrs={"aria-label": "Right arrow"} as an argument

avg = soup.find('svg',attrs={"aria-label": "Right arrow"})

# That SVG is inside of a div
svg_parent = avg.parent

# And the tag right before that div (its "previous sibling")
# is an anchor (link) tag with the path we need
link = svg_parent.previous_sibling

# Then we can extract the path from that link to build the full URL
relative_path = link['href']
next_page_url = 'https://web.archive.org' + relative_path
next_page_url

'https://web.archive.org/web/20210326225933/https://ra.co/events/us/newyork?week=2019-04-06'

In [162]:
# Fill in this function, to take in the current page's URL and return the
# next page's URL
def next_page(url):
    avg = soup.find('svg',attrs={"aria-label": "Right arrow"})
    svg_parent = avg.parent
    link = svg_parent.previous_sibling
    relative_path = link['href']
    next_page_url = 'https://web.archive.org' + relative_path
    return next_page_url

In [163]:
# Test out your function
next_page(EVENTS_PAGE_URL)

'https://web.archive.org/web/20210326225933/https://ra.co/events/us/newyork?week=2019-04-06'

## Scrape the Next 500 Events

In other words, repeatedly call `scrape_events` and `next_page` until you have assembled a dataframe with at least 500 rows.

Display the data sorted by the number of attendees, greatest to least.

We recommend adding a brief `time.sleep` call between `requests.get` calls to avoid rate limiting.

In [172]:
import time
overall_df = pd.DataFrame()
current_url = EVENTS_PAGE_URL
while len(overall_df) < 500:
    df = scrape_events(current_url)
    overall_df = pd.concat([overall_df,df],axis=0).reset_index(drop=True)
    current_url = next_page(current_url)
    time.sleep(1)


In [174]:
overall_df.sort_values('Number_of_Attendees', ascending=False)

,Event_Name,Venue,Event_Date,Number_of_Attendees
351,Zero presents... The Masquerade,The 1896,"Sat, 6 Apr",919.0
467,Zero presents... The Masquerade,The 1896,"Sat, 6 Apr",919.0
235,Zero presents... The Masquerade,The 1896,"Sat, 6 Apr",919.0
119,Zero presents... The Masquerade,The 1896,"Sat, 6 Apr",919.0
324,Stavroz live! presented by Zero,The Williamsburg Hotel,"Fri, 12 Apr",481.0
...,...,...,...,...
39,"Petra, Matthusen & Lang, White & Pitsiokos, an...",H0L0,"Sat, 30 Mar",NaN
154,Bonsai: West1ne,Sunnyvale,"Sat, 6 Apr",NaN
270,Bonsai: West1ne,Sunnyvale,"Sat, 6 Apr",NaN
386,Bonsai: West1ne,Sunnyvale,"Sat, 6 Apr",NaN


## Summary 

Congratulations! In this lab, you successfully developed a pipeline to scrape a website for concert event information!